In [1]:
import numpy as np

np.set_printoptions(precision=6)

In [12]:
def get_x(mu, Theta):
    x = np.array([1/len(mu) for _ in range(len(mu))])
    for _ in range(10000):
        x = (mu * x).dot(Theta) / mu
    return x

def get_a(L, Q, k):
    a = [[0] * L for _ in range(Q + 1)]
    a[0] = a[1] = [1] * L
    for j in range(2, Q + 1):
        for i in range(L):
            a[j][i] = min(j, k[i])
    return a

# def get_A(L, Q, a):
#     A = [[1] * L for _ in range(Q + 1)]
#     for i in range(Q + 1):
#         for j in range(L):
#             if i == 0:
#                 A[i][j] = 1
#             else:
#                 # trouble ?
#                 A[i][j] *= a[i][j]
#     return A

def get_A(L, Q, a):
    A = [[1] * L for _ in range(Q + 1)]
    for m in range(Q + 1):
        for i in range(L):
            if m == 0:
                A[m][i] = 1
            else:
                A[m][i] = A[m-1][i] * a[m][i]
    return A

def get_G(L, Q, A, x):
    G = [[0] * L for _ in range((Q + 1))]
    
    for m in range(Q + 1):
        G[m][0] = x[0]**m / A[m][0]
    G[0] = [1] * L    
    
    for y in range(Q + 1):
        for z in range(1, L):
            for m in range(y + 1):
                G[y][z] += (x[z]**m / A[m][z]) * G[y - m][z - 1] 
    return G

def function_G_m_L(m, L, Q, x, A, G):
    return (x[L]**m / A[m][L]) * G[Q - m][L - 1]
    

In [13]:
# Вариант 2
# Тестовые данные
L = 6
Q = 10
k_list = [3, 2, 2, 2, 5, 4]
mu = np.array([.2, .2, .3, .3, .4, .2])
Theta = np.array([
    [.0, .4, .0, .0, .0, .6],
    [.2, .0, .7, .0, .1, .0],
    [.6, .0, .0, .4, .0, .0],
    [.0, .5, .0, .0, .5, .0],
    [.6, .0, .7, .0, .0, .3],
    [.0, .0, .1, .8, .1, .0]
])

# L = 7
# Q = 10
# k_list = [2, 3, 3, 2, 4, 3, 2]
# mu = np.array([.5, .4, .8, 1, .5, .6, 1])
# Theta = np.array([
#     [.0, .0, .4, .3, .0, .3, .0],
#     [.0, .0, .0, .6, .3, .0, .1],
#     [.4, .2, .0, .2, .0, .2, .0],
#     [.0, .7, .1, .0, .2, .0, .0],
#     [.6, .0, .0, .2, .0, .0, .2],
#     [.0, .1, .3, .0, .1, .0, .5],
#     [.2, .0, .3, .1, .0, .4, .0],
# ])


# Решение системы линейных уравнений
x = get_x(mu, Theta)
print('\nx = ', x)

# for _ in range(L):
# Вычисление базовых функций

# Коэффициенты интенсивности обслуживания
a = get_a(L, Q, k_list)

# Вспомогательная функция
A = get_A(L, Q, a)
# Базовые функции
G = get_G(L, Q, A, x)


# стац. вер. что в системах не менее m треб.
PM = [[0] * L for _ in range((Q + 1))]
# стац. вер. что в системах ровно m треб.
Pm = [[0] * L for _ in range((Q + 1))]
# м.о. числа треб. в системах
s = [0] * L
# м.о. числа занятых приборов в системах
h = [0] * L
# м.о. числа треб. ожидающих в очереди систем 
b = [0] * L
# интенсивности входного потока треб. в системах
lambdas = [0] * L
# коэфф. использования систем
psy = [0] * L
# м.о. длит. пребывания треб. в системах
u = [0] * L
# м.о. длит. ожидания треб. в очереди систем
w = [0] * L
# м.о. длит. реакции сети обслуживания для систем
zita = [0] * L

# Вычисление стационарных характеристик
for m in range(Q + 1):
    for i in range(L):
        Pm[m][i] = function_G_m_L(m, i, Q, x, A, G) / G[m][i]
        summa = 0
        for k in range(m, Q + 1):
            summa += function_G_m_L(k, i, Q, x, A, G)
        PM[m][i] = (1 / G[m][i]) * summa

for i in range(L):
    summa_s = 0
    for m in range(1, Q + 1):
        for k in range(m, Q + 1):
            summa_s += function_G_m_L(k, i, Q, x, A, G)
    s[i] = (1 / G[Q][i]) * summa_s  


    summa_h1 = 0
    summa_h2 = 0
    for m in range(k_list[i] + 1):
        summa_h1 += m * function_G_m_L(m, i, Q, x, A, G)
    for m in range(k_list[i] + 2, Q + 1):
        summa_h2 += function_G_m_L(k, i, Q, x, A, G)
    h[i] = (1 / G[Q][i]) * (summa_h1 + k_list[i] * summa_h2)

    b[i] = s[i] - h[i]
    lambdas[i] = h[i] / mu[i] 
    psy[i] = h[i] / k_list[i]
    u[i] = s[i] / lambdas[i]
    w[i] = b[i] / lambdas[i]

    summa_zita = 0
    for j in range(L):
        if i != j:
            lambdas[j] * u[j]
    zita[i] = 1 / lambdas[i] * summa_zita
    
#     # Пункт 5: Перенумерация систем обслуживания в сети сл.обр.
#     x = [x[-1]] + list(x[0:-1])
#     print('x', x)
#     mu = [mu[-1]] + list(mu[0:-1])
#     print('mu', mu)
#     k_list = [k_list[-1]] + k_list[0:-1]
#     print('k', k_list)
#     print()

        
print('\nСтационарные вероятности пребывания в системах m или более требований\n')
print(' ' * 40, 'Номер системы')
print('Кол.треб.', ' ' * 5, '1', ' ' * 7, '2', ' ' * 7, '3', ' ' * 7, '4', ' ' * 7,'5', ' ' * 7, '6', ' ' * 7, ' 7')
for i in range(Q + 1):
    print(i, end='\t| ')
    for j in range(L):
        print('{:07}'.format(round(PM[i][j], 3)), end=' | ')
    print('\n', '-' * 88)
        
print('#' * 100)   
        
print('\nСтационарные вероятности пребывания в системах ровно m требований\n')
print(' ' * 40, 'Номер системы')
print('Кол.треб.', ' ' * 5, '1', ' ' * 7, '2', ' ' * 7, '3', ' ' * 7, '4', ' ' * 7, '5', ' ' * 7, '6', ' ' * 7, ' 7')
for i in range(Q + 1):
    print(i, end='\t| ')
    for j in range(L):
        print('{:07}'.format(round(Pm[i][j], 3)), end=' | ')
    print('\n', '-' * 88)

print('\nСтационарные характеристики сети\n')
print('Система', ' ' * 8, 'zita', ' ' * 16, 'h', ' ' * 12, 'psy', ' ' * 10, 'u', sep='')
for i in range(L):
    print(i + 1, end='\t|')
    print(round(zita[i], 3), round(h[i], 3), round(psy[i], 3), round(u[i], 3), sep=' \t|', end=' \t|\n')
    print('-' * 73)

print('\nСтационарные характеристики сети\n')
print('Система', ' ' * 8, 'w', ' ' * 16, 'b', ' ' * 12, 's', ' ' * 10, 'lambdas', sep='')
for i in range(L):
    print(i + 1, end='\t|')
    print(round(w[i], 3), round(b[i], 3), round(s[i], 3), round(lambdas[i], 3), sep=' \t|', end=' \t|\n')
    print('-' * 73)


x =  [3.725145e+286 2.956967e+286 2.371649e+286 2.225333e+286 1.044883e+286
 2.678928e+286]

Стационарные вероятности пребывания в системах m или более требований

                                         Номер системы
Кол.треб.       1         2         3         4         5         6          7
0	| 0000inf | 0000inf | 0000inf | 0000inf | 0000inf | 0000inf | 
 ----------------------------------------------------------------------------------------
1	| 0000inf | 0000inf | 0000inf | 0000inf | 0000inf | 0000inf | 
 ----------------------------------------------------------------------------------------
2	| 0000nan | 0000nan | 0000nan | 0000nan | 0000nan | 0000nan | 
 ----------------------------------------------------------------------------------------
3	| 0000nan | 0000nan | 0000nan | 0000nan | 0000nan | 0000nan | 
 ----------------------------------------------------------------------------------------
4	| 0000nan | 0000nan | 0000nan | 0000nan | 0000nan | 0000nan | 
 ---------------

<ipython-input-12-62bb0fabb47d>:40: RuntimeWarning: overflow encountered in double_scalars
  G[m][0] = x[0]**m / A[m][0]
<ipython-input-12-62bb0fabb47d>:46: RuntimeWarning: overflow encountered in double_scalars
  G[y][z] += (x[z]**m / A[m][z]) * G[y - m][z - 1]
<ipython-input-12-62bb0fabb47d>:50: RuntimeWarning: overflow encountered in double_scalars
  return (x[L]**m / A[m][L]) * G[Q - m][L - 1]
<ipython-input-13-38875c994c37>:71: RuntimeWarning: invalid value encountered in double_scalars
  Pm[m][i] = function_G_m_L(m, i, Q, x, A, G) / G[m][i]
<ipython-input-13-38875c994c37>:75: RuntimeWarning: invalid value encountered in double_scalars
  PM[m][i] = (1 / G[m][i]) * summa
<ipython-input-13-38875c994c37>:82: RuntimeWarning: invalid value encountered in double_scalars
  s[i] = (1 / G[Q][i]) * summa_s
<ipython-input-13-38875c994c37>:88: RuntimeWarning: invalid value encountered in double_scalars
  summa_h1 += m * function_G_m_L(m, i, Q, x, A, G)


In [23]:
k_list = [1, 2, 3, 4, 5, 6, 7, 8, 9] 
print(k_list)

k_list = [k_list[-1]] + k_list[0:-1]
print(k_list)


[1, 2, 3, 4, 5, 6, 7, 8, 9]
[9, 1, 2, 3, 4, 5, 6, 7, 8]
